# Exemple d'extraction de graphe

## Setup

***Load modules*** 
This will require amtrack repo in the right path I use environment variables but feel free to do something better.
Normally all the required packages are indicated, if some are missing, please add them.
If you also want to create a "freeze" or a poetry thing to have an easier installation of the environment please do so.


In [1]:
%load_ext autoreload
%autoreload 2
import os  
import sys  
import pandas as pd
from amftrack.util.sys import get_dates_datetime, get_dirname, temp_path, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id, fiji_path, path_code
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment, Edge,Node, plot_raw_plus
from amftrack.pipeline.paths.directory import run_parallel_transfer, find_state, directory_scratch, directory_project, directory_archive
import dropbox
from amftrack.transfer.functions.transfer import upload, zip_file
from subprocess import call
from tqdm.autonotebook import tqdm
from time import time_ns
import subprocess
import matplotlib as mpl
from amftrack.util.sys import temp_path

/home/ipausers/kahane/Wks/AMFtrack/amftrack/transfer/functions/transfer.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


***choose the path of interest***
I mounted sun on my linux computer at AMOLF. Please feel free to do so on yours as well. I can send you the command. 
Otherwise, if you have the folders locally, just indicate the path.
Finally you can also simply ssh to my computer and run things from there.

In [2]:
directory = "/data/felix/width1/full_plates/" # careful: must have the / at the end

update_plate_info(directory)
folder_df = get_current_folders(directory)

analysed:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
len(folder_df)

2

***Choose the folder you're interested in***

In [4]:
folder_df

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp
12704,/data/felix/width1/full_plates/20220330_0049_P...,865,7,'Carrot','A5','001P100N100C','Y','-',20220307,25,...,False,False,False,False,False,False,False,"30.03.2022, 00:49:",20220330_0049_Plate943,NaN
12705,/data/felix/width1/full_plates/20220325_1423_P...,907,9,'Carrot','A5','001P100N100C','Y','-',20220321,25,...,False,False,False,False,False,False,False,"25.03.2022, 14:23:",20220325_1423_Plate907,NaN


In [5]:
str(folder_df['total_path'].iloc[1])

'/data/felix/width1/full_plates/20220325_1423_Plate907'

In [6]:
folders = folder_df.loc[folder_df['Plate']=="907"]

In [7]:
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp
12705,/data/felix/width1/full_plates/20220325_1423_P...,907,9,'Carrot','A5','001P100N100C','Y','-',20220321,25,...,False,False,False,False,False,False,False,"25.03.2022, 14:23:",20220325_1423_Plate907,NaN


These are local adaptations of functions for running things on the supercomputer. So better to keep them in the notebook at this stage.

## Stiching function

In [10]:
from amftrack.util.sys import path_code
import os

def make_stitching_loop(directory,dirname,op_id):
    # TODO(FK): handle the case where there is no stiching_loops directory
    #MODIF a_file = open(f'{path_code}amftrack/pipeline/scripts/stitching_loops/stitching_loop.ijm',"r")
    a_file = open(os.path.join(path_code, "pipeline/scripts/stitching_loops/stitching_loop.ijm") ,"r")
    
    list_of_lines = a_file.readlines()

    list_of_lines[4] = f'mainDirectory = \u0022{directory}\u0022 ;\n'
    list_of_lines[29] = f'\t if(startsWith(list[i],\u0022{dirname}\u0022)) \u007b\n'
    file_name = f'{os.getenv("TEMP")}/stitching_loops/stitching_loop{op_id}.ijm'
    a_file = open(file_name, "w")

    a_file.writelines(list_of_lines)

    a_file.close()
    
def run_stitch(directory, folders):
    folder_list = list(folders['folder'])
    folder_list.sort()    
    with tqdm(total=len(folder_list), desc="stitched") as pbar:
        for folder in folder_list:
            op_id = time_ns()
            im = imageio.imread(f'{directory}/{folder}/Img/Img_r03_c05.tif')
            for x in range(1,11):
                for y in range(1,16):
                    strix= str(x) if x>=10 else f'0{x}'
                    striy= str(y) if y>=10 else f'0{y}'
                    path = f'{directory}/{folder}/Img/Img_r{strix}_c{striy}.tif'
                    # print(striy,path,os.path.getsize(path))
                    if not os.path.isfile(path):
                        f = open(path, 'w')
                    if os.path.getsize(path)==0:
                        imageio.imwrite(path,im*0)
            make_stitching_loop(directory,folder,op_id)
            # MODIF 2 command = [os.getenv('HOME')+'/Fiji.app/ImageJ-linux64','--mem=8000m','--headless','--ij2','--console','-macro',f'{os.getenv("TEMP")}/stitching_loops/stitching_loop{op_id}.ijm']
            command = [fiji_path,'--mem=8000m','--headless','--ij2','--console','-macro',f'{os.getenv("TEMP")}/stitching_loops/stitching_loop{op_id}.ijm']
            print(' '.join(command))
            process = subprocess.run(command)
            pbar.update(1)

In [26]:
path_code

'/home/ipausers/kahane/Wks/AMFtrack/amftrack/'

In [11]:
def run_parallel(code, args, folders):
    op_id = time_ns()
    folders.to_json(f'{os.getenv("TEMP")}/{op_id}.json')# temporary file
    folder_list = list(folders['folder'])
    folder_list.sort()  
    args_str = [str(arg) for arg in args]
    arg_str = " ".join(args_str)
    with tqdm(total=len(folder_list), desc="stitched") as pbar:
        for index,folder in enumerate(folder_list):
            command = ['python3',f'{path_code}pipeline/scripts/image_processing/{code}']+args_str+[f'{op_id}',f'{index}']
            print(' '.join(command))
            process = subprocess.run(command)
            pbar.update(1)

***Stitch the folders you're interested in***

should take about 20mins

In [9]:
run_stitch(directory,folders)

NameError: name 'run_stitch' is not defined

***Compress the image and skeletonize***

should take 1h30-2h

In [25]:
args = [directory]
run_parallel('compress_image.py',args, folders)

stitched:   0%|          | 0/1 [00:00<?, ?it/s]

python3 /home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/scripts/image_processing/compress_image.py /data/felix/width1/full_plates/ 1649864771751109735 0
Directory  /data/felix/width1/full_plates/20220325_1423_Plate907/Analysis  Created 


In [12]:
time = '3:00:00'
low = 30
high = 80
extend = 30
args=[low, high, extend,directory]
run_parallel('extract_skel.py',args,folders)

stitched:   0%|          | 0/1 [00:00<?, ?it/s]

python3 /home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/scripts/image_processing/extract_skel.py 30 80 30 /data/felix/width1/full_plates/ 1649921815530100881 0


/home/ipausers/kahane/.local/lib/python3.8/site-packages/skimage/feature/corner.py:259: RuntimeWarning: invalid value encountered in sqrt
  l2 = (M00 + M11) / 2 - np.sqrt(4 * M01 ** 2 + (M00 - M11) ** 2) / 2
/home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/functions/image_processing/extract_graph.py:169: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  graph = graph.append(pd.DataFrame(edges[branch]), ignore_index=True)
/home/ipausers/kahane/.local/lib/python3.8/site-packages/skimage/feature/corner.py:258: RuntimeWarning: overflow encountered in multiply
  l1 = (M00 + M11) / 2 + np.sqrt(4 * M01 ** 2 + (M00 - M11) ** 2) / 2
/home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/functions/image_processing/extract_graph.py:169: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  graph = graph.append(pd.DataFrame(edges[br

/home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/functions/image_processing/extract_graph.py:169: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  graph = graph.append(pd.DataFrame(edges[branch]), ignore_index=True)
/home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/functions/image_processing/extract_graph.py:169: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  graph = graph.append(pd.DataFrame(edges[branch]), ignore_index=True)
/home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/functions/image_processing/extract_graph.py:169: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  # TODO(FK): Use pandas.concat instead (Frame.append soon deprecated)
/home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/functions/image_processing/ex

/home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/functions/image_processing/extract_graph.py:169: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  # TODO(FK): Use pandas.concat instead (Frame.append soon deprecated)
/home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/functions/image_processing/extract_graph.py:169: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  # TODO(FK): Use pandas.concat instead (Frame.append soon deprecated)
/home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/functions/image_processing/extract_graph.py:169: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  # TODO(FK): Use pandas.concat instead (Frame.append soon deprecated)
/home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/functions/image_processing/ex

Directory  /data/felix/width1/full_plates/20220325_1423_Plate907/Analysis  already exists
number to reduce :  17640067 1399746433
time= 4281.725394487381


## Visualisation

***Visualise the skeletonization***

In [ ]:
%matplotlib inline
import mpld3
mpld3.enable_notebook()

In [ ]:
select = folders
folder_list = list(select['total_path'])
folder_list.sort()
skels = []
ims = []
kernel = np.ones((5,5),np.uint8)
itera = 1
start = 0

finish = start+2
for folder in folder_list[start:finish]:
    path_snap=folder
    skel_info = read_mat(path_snap+'/Analysis/skeleton_compressed.mat')
    skel = skel_info['skeleton']
    skels.append(cv2.dilate(skel.astype(np.uint8),kernel,iterations = itera))
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    ims.append(im)

In [ ]:
plt.close('all')
start = 0
finish = start + 1
for i in range(start,finish):
    plot_t_tp1([], [], None, None, skels[i], ims[i])

***These steps may be a bit long, but it also helps just keeping things consistent and use one single framework,*** you may bypass them by artificially renaming skeleton.mat into skeleton_realigned.mat which is necessary for the next step because you probably don't care about these two steps since you're doing only one image...

In [15]:
time = '2:00'
thresh = 50
args=[thresh, directory]
run_parallel('mask_skel.py',args,folders)

stitched:   0%|          | 0/1 [00:00<?, ?it/s]

python3 /home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/scripts/image_processing/mask_skel.py 50 /data/felix/width1/full_plates/ 1649945483289864839 0
Directory  /data/felix/width1/full_plates/20220325_1423_Plate907/Analysis  already exists


In [23]:
time = '3:00:00'
threshold = 0.1
args = [threshold, directory]
run_parallel('prune_skel.py',args,folders)

stitched:   0%|          | 0/1 [00:00<?, ?it/s]

python3 /home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/scripts/image_processing/prune_skel.py 0.1 /data/felix/width1/full_plates/ 1649949381831900868 0


In [24]:
plates = list(set(list(folders['Plate'].values)))
for plate in plates:
    time = '2:00:00'
    args = [directory]
    run_parallel('realign.py', args, folders)

stitched:   0%|          | 0/1 [00:00<?, ?it/s]

python3 /home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/scripts/image_processing/realign.py /data/felix/width1/full_plates/ 1649949541803624321 0


Traceback (most recent call last):
  File "/home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/scripts/image_processing/realign.py", line 36, in <module>
    skel_info = read_mat(path_snap + "/Analysis/skeleton_pruned.mat")
  File "/home/ipausers/kahane/.local/lib/python3.8/site-packages/pymatreader/pymatreader.py", line 81, in read_mat
    raise IOError('The file %s does not exist.' % (filename,))
OSError: The file /data/felix/width1/full_plates/20220325_1423_Plate907/Analysis/skeleton_pruned.mat does not exist.


Now you're finally getting to the point where you generate the graph object

In [25]:
time = '2:00:00'
args=[directory]
run_parallel('extract_nx_graph.py',args, folders)

stitched:   0%|          | 0/1 [00:00<?, ?it/s]

python3 /home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/scripts/image_processing/extract_nx_graph.py /data/felix/width1/full_plates/ 1649949582746696916 0


Traceback (most recent call last):
  File "/home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/scripts/image_processing/extract_nx_graph.py", line 31, in <module>
    skel = read_mat(path_snap + "/Analysis/skeleton_pruned_realigned.mat")["skeleton"]
  File "/home/ipausers/kahane/.local/lib/python3.8/site-packages/pymatreader/pymatreader.py", line 81, in read_mat
    raise IOError('The file %s does not exist.' % (filename,))
OSError: The file /data/felix/width1/full_plates/20220325_1423_Plate907/Analysis/skeleton_pruned_realigned.mat does not exist.


***and now you have the Experiment object to play with and query images related to position of edges etc...***


In [21]:
run_info = folders
plate = int(list(run_info['folder'])[0].split('_')[-1][5:])
folder_list = list(run_info['folder'])
directory_name = folder_list[0]
exp = Experiment(plate,directory)
exp.load(run_info.loc[run_info['folder']==directory_name],labeled=False)
path_snap = directory + directory_name
(G, pos) = exp.nx_graph[0],exp.positions[0]

2022-03-25 14:23:00


FileNotFoundError: [Errno 2] No such file or directory: '/data/felix/width1/full_plates/20220325_1423_Plate907/Analysis/nx_graph_pruned.p'

In [ ]:
from random import choice
edge = choice(list(G.edges))

In [ ]:
edge_exp = Edge(Node(edge[0],exp),Node(edge[1],exp),exp)

***Visualiser l'image correspondant à l'edge*** 

***Visualiser tout le réseau avec l'edge idniqué dessus***

## Brouillon

In [ ]:
exp

In [ ]:
dir(exp)